In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import time
import datetime as dt

In [ ]:
temp=pd.read_csv('/kaggle/input/ecommerce-data/data.csv')
df= pd.read_csv('/kaggle/input/ecommerce-data/data.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
(df.isnull().sum())


> ***Since CustomerID is missing ,there is no known way to find it .It is best to drop all those rows* .**

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.nunique()

In [ ]:
df.info()

In [ ]:
df= df[df['Quantity']>=0]
df = df[df['UnitPrice']>=0]

In [ ]:
df.describe()

In [ ]:
df['totalcost']=df['Quantity']*df['UnitPrice']
df

In [ ]:
pi=df.groupby('Country').count().sort_values("InvoiceNo",ascending = False)
pi.head()

In [ ]:

f=df['Country'].unique()

fig = plt.figure(figsize =(10, 7)) 
plt.pie(pi['InvoiceNo'].head(6), labels= f[:6],autopct='%1.2f%%',explode = [0.2, 0, 0, 0,0,0])
plt.title("Country wise Distribution")
plt.show() 


In [ ]:
p=df.groupby('Description').count().sort_values("InvoiceNo",ascending = False)
p=p.head(15)

sns.barplot(p.index,p['Quantity'],palette = "rocket")
plt.xlabel('Top Selling Items')
plt.xticks(rotation=90)


In [ ]:

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Date'] = df['InvoiceDate'].apply(lambda x: x.date())
df.head()


#df['Date'] = df['InvoiceDate'].dt.date
#df.head()

In [ ]:
recency_df = df.groupby(by='CustomerID', as_index=False)['Date'].max()
recency_df.columns = ['CustomerID','LastPurchaseDate']
recency_df

In [ ]:
now = dt.date(2021,12,1)
print(now)

In [ ]:
recency_df['Recency'] = recency_df['LastPurchaseDate'].apply(lambda x: (now - x).days)


recency_df.drop('LastPurchaseDate',axis = 1,inplace=True)
recency_df.head(5)

In [ ]:
temp = df.copy()
temp.drop_duplicates(['InvoiceNo','CustomerID'],inplace=True)
frequency_df = temp.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

In [ ]:
monetary_df = df.groupby(by = 'CustomerID',as_index=False).agg({'totalcost':'sum'})
monetary_df.columns = ['CustomerID','TotalCost']
monetary_df.head(5)

In [ ]:
rfm = recency_df.merge(frequency_df,on='CustomerID').merge(monetary_df,on='CustomerID')
rfm.set_index('CustomerID',inplace=True)
a=rfm.columns


In [ ]:
print(rfm.corr())
sns.heatmap(rfm.corr(),cmap="RdYlGn",annot=True)

In [ ]:
sns.pairplot(rfm, diag_kind="hist")

In [ ]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()
rfm = pd.DataFrame(pt.fit_transform(rfm))
rfm.columns = a
rfm.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
rfm_scaled = sc.fit_transform(rfm)
rfm_scaled[:5]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca_tranformed_data = pca.fit_transform(rfm_scaled)

In [ ]:
X = rfm.copy()
pca = PCA(n_components = 2)
df_pca = pca.fit_transform(X)

df_pca = pd.DataFrame(df_pca)
df_pca.head(5)

In [ ]:
X=df_pca.copy()

In [ ]:
from sklearn.cluster import KMeans 

cluster_range = range(1, 8)
cluster_errors = []
cluster_sil_scores = []

for num in cluster_range: 
    clusters = KMeans(num, n_init = 100,init='k-means++',random_state=0)
    clusters.fit(X)
    labels = clusters.labels_                     # capture the cluster lables
    centroids = clusters.cluster_centers_         # capture the centroids
    cluster_errors.append( clusters.inertia_ )    # capture the intertia
clusters_df = pd.DataFrame({ "num_clusters":cluster_range, "cluster_errors": cluster_errors} )
clusters_df[0:10]

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(clusters_df["num_clusters"],clusters_df["cluster_errors"],marker = 'o')
plt.xlabel('count of clusters')
plt.ylabel('error')

In [ ]:
from  sklearn.metrics import silhouette_score

for num in range(2,16):
    clusters = KMeans(n_clusters=num,random_state=0)
    labels = clusters.fit_predict(df_pca)
    
    sil_avg = silhouette_score(df_pca, labels)
    print('For',num,'The Silhouette Score is =',sil_avg)

In [ ]:
kmeans = KMeans(n_clusters = 4,random_state=0)
kmeans = kmeans.fit(df_pca)
labels = kmeans.predict(df_pca)
centroids = kmeans.cluster_centers_

In [ ]:
df_pca['Clusters'] = labels
df_pca.head()

In [ ]:
df_pca['Clusters'].value_counts()

In [ ]:
sns.pairplot(df_pca,diag_kind='hist',hue='Clusters',palette='rocket')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score


In [ ]:
X = df_pca[[0,1]]
Y = df_pca['Clusters']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)
lr = LogisticRegression(max_iter=1000,random_state=0)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('Test accuracy = ', accuracy_score(y_test, y_pred))